
### Entrada
- x1: Local da Coleta em km
- x2: Nivel da Mare em m
- x3: Umidade do Ar (%)
- x4: Índice Pluvimétrico (mm)¹
- x5: Temperatura (°C)

### Saida
- y1: pH
- y2: Condutividade Elétrica (mS.cmˉ¹)
- y3: Oxigênio Dissolvido (mg.Lˉ¹)
- y4: Sólidos Totais Dissolvidos (ppm)


In [1]:
!pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [2]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

Cloning into 'tf-levenberg-marquardt'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 53 (delta 31), reused 29 (delta 14), pack-reused 0
Receiving objects: 100% (53/53), 32.99 KiB | 11.00 MiB/s, done.
Resolving deltas: 100% (31/31), done.


In [3]:
import numpy as np
import pandas as pd

def create_dataframe(output, label):
    df = pd.read_excel("../../../data.xlsx")
    x1, x2, x3, x4, x5  = df['x1'], df['x2'], df['x3'], df['x4'], df['x5']

    input = np.vstack([x1, x2, x3, x4, x5 ]).T
    output = np.array(df[output])
    return output, input

output, input = create_dataframe(output="y1", label="ph")

/tmp/ipykernel_2403236/4275043310.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
import tensorflow as tf
import numpy as np
from keras import regularizers
from keras import initializers
import levenberg_marquardt as lm

# layers, neurons
class ShuffleArchitecture:
    def __init__(self, input_size, hidden_sizes, output_size, act_h, act_o, param_reg):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.act_h = act_h
        self.act_o = act_o
        self.regularizer = regularizers.L2(param_reg)
        self.initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

    def set_architecture(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(self.hidden_sizes[0],
                        input_shape=(self.input_size,),
                        activation=self.act_h,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,                        
                        ))  # input layer
        
        for size in self.hidden_sizes[1:]:  # hidden layers
            self.model.add(tf.keras.layers.Dense(size,
                            activation=self.act_h,
                            kernel_regularizer=self.regularizer,
                            kernel_initializer=self.initializer,  
                        ))

        self.model.add(tf.keras.layers.Dense(self.output_size,
                        activation=self.act_o,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,  
                        ))  # output layer

    def create_model(self, _learning_rate):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=_learning_rate),
            loss=tf.keras.losses.MeanSquaredError())

        self.lm_model = lm.ModelWrapper(
            tf.keras.models.clone_model(self.model))

        self.lm_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=_learning_rate),
            loss=lm.MeanSquaredError())
        return(self.lm_model)

2024-03-24 19:17:21.346899: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-24 19:17:21.382553: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 19:17:21.382596: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 19:17:21.384066: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-24 19:17:21.391203: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-24 19:17:21.391715: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error


class TrainWithSmallDataset:
    def __init__(self, batch_size=1000):
        self.batch_size = batch_size
        self.betters = []
    
    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)

    def split_dataset(self, input, output):
      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)
 
 
    
    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)

          metrics = {
                          'r2': r2,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics

In [6]:
from itertools import product
import pickle

class Tester:
  def __init__(self, input, output, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input, self.output = input, output
  
  def setArchitecure(self, trainer, _hidden_sizes, _pg, _lr):
    shuffler = ShuffleArchitecture(input_size=5,
                                    hidden_sizes=_hidden_sizes,
                                    output_size=1,
                                    act_h='tanh',
                                    act_o='linear',
                                    param_reg=_pg)
    shuffler.set_architecture()    
    trainer.lm_model = shuffler.create_model(_lr)

  def Train(self, trainer, epochs=1000):
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path) 

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)


  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ [{idx}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] >= boundarie): # should be <= to descending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

# Init
  def Loop(self, sort_by, boundarie, hidden_sizes, regularizers, learning_rate):
    trainer = TrainWithSmallDataset()

    for count, (hidden_size, reg, lr) in enumerate(product(hidden_sizes, regularizers, learning_rate), start=1):
      header =  f"Hidden Size={hidden_size}, regularizer={reg}, learning_rate={lr}"
      print(f"Testando combinacao{count}: {header}")
      self.setArchitecure(trainer, hidden_size, reg, lr)
      for j in range(self.dataset_run_times):
        trainer.split_dataset(self.input, self.output)
        if (self.LoopWeights(sort_by, boundarie, trainer, f"{count}_{j}") == True):
          self.SaveDataset(trainer, f"dataset_{count}_{j}")
          self.DisplayBetterResults(sort_by, header, f"{count}_{j}")
        self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, header, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    path = f'../results/metrics_{dataset}'
    df.to_excel(f"{path}.xlsx", index=True)
    print(f"DataFrame salvo em {path}")
    with open(f"{path}.txt", 'w') as arquivo:
      arquivo.write(header)

In [7]:
tester = Tester(input, output, run_times=25, dataset_run_times=10)

In [8]:
tester.Loop(sort_by='r2',
            boundarie=0.8,
            hidden_sizes = [[55, 27]],
            regularizers=[0.02, 0.1],
            learning_rate=[0.02, 0.1])

Testando combinacao1: Hidden Size=[55, 27], regularizer=0.02, learning_rate=0.02
+++++++++++ [1_0] | 1 ++++++++++++++++++
Elapsed time:  1.2475549450009566
Stopped at epoch:  17
1/1 [==============================] - 0s 16ms/step
+++++++++++ [1_0] | 2 ++++++++++++++++++
Elapsed time:  1.6587251640012255
Stopped at epoch:  20
1/1 [==============================] - 0s 73ms/step
+++++++++++ [1_0] | 3 ++++++++++++++++++
Elapsed time:  0.8653898169995955
Stopped at epoch:  6
1/1 [==============================] - 0s 29ms/step
+++++++++++ [1_0] | 4 ++++++++++++++++++
Elapsed time:  0.5102456870008609
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [1_0] | 5 ++++++++++++++++++
Elapsed time:  0.3800620309993974
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 6 ++++++++++++++++++
Elapsed time:  0.2815305479998642
Stopped at epoch:  6
1/1 [==============================] - 0s 21ms/step
+++++++++++ [1_0] | 7 +++++++

2024-03-24 19:22:30.214114: W tensorflow/core/data/root_dataset.cc:342] Optimization loop failed: CANCELLED: Operation was cancelled


Elapsed time:  0.2517576169993845
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/step
+++++++++++ [3_4] | 1 ++++++++++++++++++
Elapsed time:  0.2926084919999994
Stopped at epoch:  8
1/1 [==============================] - 0s 24ms/step
+++++++++++ [3_4] | 2 ++++++++++++++++++
Elapsed time:  0.22388643999875057
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [3_4] | 3 ++++++++++++++++++
Elapsed time:  0.18079829099951894
Stopped at epoch:  6
1/1 [==============================] - 0s 21ms/step
+++++++++++ [3_4] | 4 ++++++++++++++++++
Elapsed time:  0.27674521900007676
Stopped at epoch:  6
1/1 [==============================] - 0s 17ms/step
+++++++++++ [3_4] | 5 ++++++++++++++++++
Elapsed time:  0.23424608700042882
Stopped at epoch:  6
1/1 [==============================] - 0s 21ms/step
+++++++++++ [3_4] | 6 ++++++++++++++++++
Elapsed time:  0.23167331100012234
Stopped at epoch:  6
1/1 [==============================] - 0s 20ms/step


,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_3_8_0,0.831677,0.481605,0.207573,0.348368,0.031489,0.056309,0.106362,0.079864


DataFrame salvo em ../results/metrics_3_8
+++++++++++ [3_9] | 1 ++++++++++++++++++
Elapsed time:  0.5271883210007218
Stopped at epoch:  12
1/1 [==============================] - 0s 14ms/step
+++++++++++ [3_9] | 2 ++++++++++++++++++
Elapsed time:  0.22383552899918868
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [3_9] | 3 ++++++++++++++++++
Elapsed time:  0.30672507400049653
Stopped at epoch:  6
1/1 [==============================] - 0s 17ms/step
+++++++++++ [3_9] | 4 ++++++++++++++++++
Elapsed time:  0.23550240700024005
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [3_9] | 5 ++++++++++++++++++
Elapsed time:  0.1837701169988577
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [3_9] | 6 ++++++++++++++++++
Elapsed time:  0.23103200399964408
Stopped at epoch:  6
1/1 [==============================] - 0s 22ms/step
+++++++++++ [3_9] | 7 ++++++++++++++++++
Elapsed time:  0.2689094

...

DataFrame salvo em ../results/metrics_4_7
+++++++++++ [4_8] | 1 ++++++++++++++++++
Elapsed time:  0.39909603700107255
Stopped at epoch:  9
1/1 [==============================] - 0s 19ms/step
+++++++++++ [4_8] | 2 ++++++++++++++++++
Elapsed time:  0.27276648500082956
Stopped at epoch:  6
1/1 [==============================] - 0s 17ms/step
+++++++++++ [4_8] | 3 ++++++++++++++++++
Elapsed time:  0.24617356400085555
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [4_8] | 4 ++++++++++++++++++
Elapsed time:  0.2564106919999176
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [4_8] | 5 ++++++++++++++++++
Elapsed time:  0.25772265900013736
Stopped at epoch:  6
1/1 [==============================] - 0s 19ms/step
+++++++++++ [4_8] | 6 ++++++++++++++++++
Elapsed time:  0.28342613500171865
Stopped at epoch:  6
1/1 [==============================] - 0s 17ms/step
+++++++++++ [4_8] | 7 ++++++++++++++++++
Elapsed time:  0.2276109

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_4_8_18,0.903116,0.853725,0.365023,0.521269,0.018125,0.010749,0.114502,0.059574
model_4_8_17,0.903189,0.850244,0.367491,0.521864,0.018111,0.011005,0.114057,0.059500
model_4_8_16,0.903558,0.846641,0.372247,0.523981,0.018042,0.011269,0.113200,0.059236
model_4_8_15,0.904327,0.843349,0.379859,0.528142,0.017898,0.011511,0.111827,0.058719
model_4_8_14,0.905547,0.840590,0.390617,0.534617,0.017670,0.011714,0.109887,0.057913
model_4_8_13,0.907263,0.838924,0.404628,0.543650,0.017349,0.011836,0.107360,0.056789
model_4_8_20,0.909220,0.870842,0.399954,0.550441,0.016983,0.009491,0.108203,0.055944
model_4_8_19,0.909220,0.870843,0.399959,0.550444,0.016983,0.009491,0.108202,0.055943
model_4_8_24,0.909223,0.870833,0.399979,0.550455,0.016982,0.009492,0.108199,0.055942
model_4_8_22,0.909223,0.870833,0.399979,0.550455,0.016982,0.009492,0.108199,0.055942


DataFrame salvo em ../results/metrics_4_8
+++++++++++ [4_9] | 1 ++++++++++++++++++
Elapsed time:  0.04668521200073883
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [4_9] | 2 ++++++++++++++++++
Elapsed time:  0.03614823200041428
Stopped at epoch:  0
1/1 [==============================] - 0s 21ms/step
+++++++++++ [4_9] | 3 ++++++++++++++++++
Elapsed time:  0.043271080998238176
Stopped at epoch:  0
1/1 [==============================] - 0s 16ms/step
+++++++++++ [4_9] | 4 ++++++++++++++++++
Elapsed time:  0.041545676000168896
Stopped at epoch:  0
1/1 [==============================] - 0s 17ms/step
+++++++++++ [4_9] | 5 ++++++++++++++++++
Elapsed time:  0.04483570999946096
Stopped at epoch:  0
1/1 [==============================] - 0s 23ms/step
+++++++++++ [4_9] | 6 ++++++++++++++++++
Elapsed time:  0.029546902000220143
Stopped at epoch:  0
1/1 [==============================] - 0s 22ms/step
+++++++++++ [4_9] | 7 ++++++++++++++++++
Elapsed time:  0.030

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_4_9_0,0.909223,0.973238,0.988722,0.982264,0.016982,0.006864,0.004124,0.005575
model_4_9_22,0.909223,0.973238,0.988722,0.982264,0.016982,0.006864,0.004124,0.005575
model_4_9_21,0.909223,0.973238,0.988722,0.982264,0.016982,0.006864,0.004124,0.005575
model_4_9_20,0.909223,0.973238,0.988722,0.982264,0.016982,0.006864,0.004124,0.005575
model_4_9_19,0.909223,0.973238,0.988722,0.982264,0.016982,0.006864,0.004124,0.005575
model_4_9_18,0.909223,0.973238,0.988722,0.982264,0.016982,0.006864,0.004124,0.005575
model_4_9_17,0.909223,0.973238,0.988722,0.982264,0.016982,0.006864,0.004124,0.005575
model_4_9_16,0.909223,0.973238,0.988722,0.982264,0.016982,0.006864,0.004124,0.005575
model_4_9_15,0.909223,0.973238,0.988722,0.982264,0.016982,0.006864,0.004124,0.005575
model_4_9_14,0.909223,0.973238,0.988722,0.982264,0.016982,0.006864,0.004124,0.005575


DataFrame salvo em ../results/metrics_4_9
